In [1]:
import sys
import warnings
import pandas as pd

sys.path.append('../..')
from sklearn.metrics import confusion_matrix
from utils import UnimapResult, set_plot_theme

set_plot_theme()
warnings.filterwarnings('ignore')

In [2]:
result = UnimapResult('pbmc9', f'scgpt_zeroshot/2023', root_dir='../../results', detailed=False)

In [3]:
cm = confusion_matrix(result.t_result['celltype'], result.t_result['pred_celltype'])

celltype_labels = sorted(set(result.t_result['celltype']) | set(result.t_result['pred_celltype']))
cm = pd.DataFrame(cm, index=celltype_labels, columns=celltype_labels)
# cm = cm.loc[:, (cm >= 0.01).any(axis=0)]

celltype_index = ['monocyte', 'T cell', 'natural killer cell', 'B cell', 'mast cell', 'alveolar macrophage', 'plasma cell', 'dendritic cell']
pred_celltype_col = ['classical monocyte', 'macrophage', 'CD4-positive, alpha-beta T cell', 'CD16-positive, CD56-dim natural killer cell, human', 'central memory CD8-positive, alpha-beta T cell', 'CD8-positive, alpha-beta T cell', 'plasma cell', 'unswitched memory B cell', 'mast cell', 'T cell', 'monocyte', 'B cell', 'CD4-positive, alpha-beta memory T cell', 'natural killer cell', 'naive B cell', 'CD14-positive, CD16-positive monocyte', 'basophil mast progenitor cell', 'effector memory CD8-positive, alpha-beta T cell', 'CD8-positive, alpha-beta memory T cell', 'myelocyte', 'non-classical monocyte', 'gamma-delta T cell', 'IgA plasma cell', 'plasmacytoid dendritic cell', 'mature NK T cell', 'dendritic cell', 'conventional dendritic cell', 'leukocyte', 'neutrophil', 'innate lymphoid cell', 'class switched memory B cell', 'activated CD8-positive, alpha-beta T cell', 'myeloid cell', 'native cell', 'memory B cell', 'alveolar macrophage', 'CD4-positive helper T cell', 'CD16-negative, CD56-bright natural killer cell, human', 'mesenchymal cell', 'CD1c-positive myeloid dendritic cell', 'hematopoietic cell', 'capillary endothelial cell', 'erythrocyte', 'endothelial cell of vascular tree', 'erythroid lineage cell', 'plasmablast', 'effector memory CD4-positive, alpha-beta T cell', 'epithelial cell of lower respiratory tract', 'follicular B cell', 'regular ventricular cardiac myocyte', 'mural cell', 'early promyelocyte', 'basal cell', 'T follicular helper cell', 'immature B cell', 'CD14-low, CD16-positive monocyte', 'CD14-positive monocyte', 'CD8-positive, alpha-beta cytotoxic T cell', 'alpha-beta T cell']


cm = cm.reindex(index=celltype_index,
                columns=pred_celltype_col
                )
nodes = pd.DataFrame(index=[i + ' ' for i in celltype_index] + pred_celltype_col, columns=['name'])                
nodes['name'] =  [i + ' ' for i in celltype_index] + pred_celltype_col

links = pd.DataFrame(columns=['source', 'target', 'value'])
for index in celltype_index:
    for col in pred_celltype_col:
        links = links._append({'source': index + ' ', 'target': col, 'value': cm.loc[index, col]}, ignore_index=True)
links = links[links['value'] > 0]

nodes_dict = {name: idx for idx,name in enumerate(nodes.index)}
nodes.index = nodes.index.map(nodes_dict)
links['source'] = links['source'].map(nodes_dict)
links['target'] = links['target'].map(nodes_dict)

nodes.to_csv('results/nodes.csv')
links.to_csv('results/links.csv')